In [105]:
import re

import numpy as np
import pandas as pd

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [106]:
base_data = pd.read_pickle('./01_base_data.pkl')
base_data[0:2]

,Category,Scientific name,Helps,Helped by,Attracts,-Repels/+distracts,Avoid,Comments,combined_name
Common name,,,,,,,,,
allium,vegetable,allium,"fruit tree, nightshade,tomato, capsicum pepp...","carrots, tomato, carrots,african spider plan...",thrips,"rabbit, slug,see garlic, aphid, carrot fly,...","bean, pea",allium be a family of plant which include onio...,allium | allium
asparagus,vegetable,asparagus officinalis,"tomato, parsley","aster family flower, dill, coriander, tomat...",couple with basil seem to encourage lady bug,,"onion, garlic, potato, gladiolus",,asparagus officinalis | asparagus


In [107]:
# get pest names
nested = [x.split(',') for x in base_data['-Repels/+distracts']]
flattened_list = [y.strip() for x in nested for y in x if y != '']
pest_names = list(set(flattened_list))

In [108]:
am_len = len(base_data) + len(pest_names)
index = list(base_data.index) + pest_names
index.sort()
am = pd.DataFrame(np.zeros(shape=(am_len, am_len)),
                  columns=index,
                  index=index)

In [109]:
for index, row in base_data.iterrows():
    helps = [x.rstrip().lstrip() for x in row['Helps'].split(',') if x != '']
    helped_by = [x.rstrip().lstrip() for x in row['Helped by'].split(',') if x != '']
    hurt_by = [x.rstrip().lstrip() for x in row['Avoid'].split(',') if x != '']
    
    attracts = [x.rstrip().lstrip() for x in row['Attracts'].split(',') if x != '']

    repels_and_distracts = [x.rstrip().lstrip() for x in row['-Repels/+distracts'].split(',') if x != '']    
    
    for x in helps:
        if x in am.index:
            am.at[index, x] += 1
    
    for x in helped_by:
        if x in am.index:
            am.at[x, index] += 1
    
    for x in hurt_by:
        if x in am.index:
            am.at[x, index]  -= 1
    
    for x in repels_and_distracts:
        if x in am.index:
            am.at[index, x] -= 1
    

In [110]:
am.shape

(156, 156)

In [111]:
am.index

Index(['alfalfa', 'allium', 'alyssum', 'anise', 'ant', 'aphid', 'apple',
       'apricot', 'asparagus', 'asparagus beetle',
       ...
       'tomato worm', 'turnip', 'various pest', 'walnut tree', 'white fly',
       'whitefly', 'wireworm', 'wormwood', 'yarrow', 'zinnia'],
      dtype='object', length=156)

In [112]:
am

,alfalfa,allium,alyssum,anise,ant,aphid,apple,apricot,asparagus,asparagus beetle,baby 's breath,basil,bean beetle,"bean, bush","bean, fava","bean, pole",bee,bee balm,beet,beet leaf hopper,black flea beetle,blueberry,borage,brassicas,broccoli,brussels sprout,butterfly,cabbage,cabbage butterfly,cabbage fly,cabbage looper,cabbage maggot,cabbage moth larvae,cabbage root fly,cabbage weevil,cabbage worm,californian poppy,caraway,carrot fly,carrots,catnip,cauliflower,celery,chamomile,chard,chervil,chive,cilantro / coriander,colorado potato beetle,corn / maize,...,pepper,peppermint,petunia,phacelia,potato,potato beetle,pumpkin,rabbit,raccoon,radish,red spider mite,repel many bean parasite,rise,root - knot nematode,rosemary,rutabegas,sage,see garlic,slug,southernwood,soybean,spearmint,spider mite,spinach,squash,squash bug,sting nettle,strawberry,stripe cucumber beetle,stripe pumpkin beetle,summer savoury,sunflow,swan plant,sweet pea,sweet potato,tansy,tarragon,thrip,thyme,tomato,tomato worm,turnip,various pest,walnut tree,white fly,whitefly,wireworm,wormwood,yarrow,zinnia
alfalfa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
allium,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alyssum,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
anise,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
whitefly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wireworm,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [113]:
def affects_growth(common_plant_name):
    return am[common_plant_name][am[common_plant_name]!=0.0]

def growth_affected_by(common_plant_name):
    return am.loc[common_plant_name][am.loc[common_plant_name] != 0.0]

In [114]:
affects_growth('allium').to_dict()

{'carrots': 3.0,
 'marigold': 2.0,
 'pansy': 1.0,
 'pea': -1.0,
 'peppermint': 1.0,
 'spearmint': 1.0,
 'tomato': 1.0}

In [115]:
growth_affected_by('allium').to_dict()

{'aphid': -1.0,
 'brassicas': 2.0,
 'cabbage looper': -1.0,
 'cabbage maggot': -1.0,
 'cabbage worm': -1.0,
 'carrot fly': -1.0,
 'carrots': 2.0,
 'fruit tree': 2.0,
 'japanese beetle': -1.0,
 'legumes': -1.0,
 'nightshade': 2.0,
 'parsley': -1.0,
 'potato': 1.0,
 'rabbit': -1.0,
 'see garlic': -1.0,
 'slug': -1.0,
 'tomato': 2.0}

In [116]:
growth_affected_by('cucumber').to_dict()

{'ant': -1.0,
 'kohlrabi': 2.0,
 'legumes': 1.0,
 'lettuce': 2.0,
 'potato': -1.0,
 'raccoon': -1.0}

In [84]:
am.to_pickle('./02_adjacency_matrix.pkl')